In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [8]:
df = pd.read_csv('/Users/localadmin/Desktop/bda datasets/cleaned_multiclass_dataset.csv')
df.head()

,cleaned_text,label
0,"['word', 'food', 'crapilicious']",0
1,['white'],0
2,"['classy', 'whore', 'red', 'velvet', 'cupcake']",0
3,"['meh', 'p', 'thanks', 'head', 'concerned', 'a...",0
4,"['isi', 'account', 'pretending', 'kurdish', 'a...",0


In [9]:
x= df['cleaned_text']
y= df['label']

In [10]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3,stratify=y, random_state=42)
print(x_train.shape,x_test.shape, y_train.shape,y_test.shape)

(33384,) (14308,) (33384,) (14308,)


In [11]:
from keras.preprocessing import text, sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

max_length = 5000
max_features = 20000
embedding_dim = 300

x_all = []
x_all.extend(x_test)
x_all.extend(x_train)

tk = Tokenizer(num_words=max_features, lower=True, filters='\n\t')
tk.fit_on_texts(x_all)
x_train_seq = tk.texts_to_sequences(x_train)
x_test_seq = tk.texts_to_sequences(x_test)

np_x_train = pad_sequences(x_train_seq, maxlen=max_length,  padding='post')
np_x_test = pad_sequences(x_test_seq, maxlen=max_length,  padding='post')
np_y_train = to_categorical(y_train)

class_num = np_y_train.shape[1]

print ('np_x_train shape: {}'.format(np_x_train.shape))
print ('np_x_test shape: {}'.format(np_x_test.shape))
print ('np_y_train shape: {}'.format(np_y_train.shape))

np_x_train shape: (33384, 5000)
np_x_test shape: (14308, 5000)
np_y_train shape: (33384, 6)


In [ ]:
from keras.models import Model
from keras.layers import *
from keras.callbacks import EarlyStopping

def one_input_classifier(max_length, max_features, embedding_dim, class_num):
    inputs = Input(shape=(max_length,), name='input_1')
    embeddings = Embedding(max_features, embedding_dim, input_length=max_length, name='embedding_1')(inputs)

    conv_1 = Conv1D(32, 9, activation='relu', name='conv1d_1')(embeddings)
    maxpool_1 = MaxPooling1D(16, name='maxpool1d_1')(conv_1)
    dropout_1 = Dropout(0.2, name='dropout_1')(maxpool_1)

    conv_2 = Conv1D(32, 7, activation='relu', name='conv1d_2')(dropout_1)
    maxpool_2 = MaxPooling1D(8, name='maxpool1d_2')(conv_2)
    dropout_2 = Dropout(0.2, name='dropout_2')(maxpool_2)

    bilstm = Bidirectional(LSTM(32, dropout=0.2, recurrent_dropout=0.2, name='lstm_1'),
        name='bidirectional_1')(dropout_2)
    preds = Dense(class_num, activation='softmax', name='preds')(bilstm)

    model = Model(inputs=inputs, outputs=preds)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

models = []
classifier_num = 10

for i in range(classifier_num):
    model = one_input_classifier(max_length, max_features, embedding_dim, class_num)

    if i == 0:
        print(model.summary())

    model.fit(np_x_train, np_y_train, validation_split=0.3, shuffle=True, epochs=5, batch_size=32, verbose=1)
    models.append(model)

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 5000)]            0         
                                                                 
 embedding_1 (Embedding)     (None, 5000, 300)         6000000   
                                                                 
 conv1d_1 (Conv1D)           (None, 4992, 32)          86432     
                                                                 
 maxpool1d_1 (MaxPooling1D)  (None, 312, 32)           0         
                                                                 
 dropout_1 (Dropout)         (None, 312, 32)           0         
                                                                 
 conv1d_2 (Conv1D)           (None, 306, 32)           7200      
                                                                 
 maxpool1d_2 (MaxPooling1D)  (None, 38, 32)            0   